## What to do?

Just run the cells that you need.
There's different sections for the different model architectures

In [1]:
import numpy as np
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.utils import to_categorical, plot_model
from time import time
from models import ResNet


Using TensorFlow backend.


import training and validation data
not included in repo, set your own folder location

In [ ]:
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_val = np.load('X_val.npy')
Y_val = np.load('Y_val.npy')

The training data is very unbalanced with a much larger 'unknown' class.
keras has a class_weight paramter in the fit() function that will give higher penalties to missclassifications in underrepresented classes. It takes a dict like this:

In [ ]:
classes = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']



some constants we need for all models

In [ ]:
input_size = X_train.shape[1:]
batch_size = 196

### First the ResNet
declare filters for each block of blocks and set output size.

In [2]:
filters_list = [8,16,32]
output_size = 12

#adjust these strings for organizeing the saved files
date = '1301'
arch = 'resnet8_16_32'

Build the model

In [ ]:
sr = ResNet(filters_list, input_size, output_size)
sr.build()
sr.m.compile(loss='categorical_crossentropy', 
             optimizer='adadelta', 
             metrics=["accuracy"])

In [ ]:
#callbacks, remember to make folders to store files 
checkpointer = ModelCheckpoint(filepath="./models/{}_{}.h5".format(arch, date),
                               verbose=0,
                               save_best_only=True)
   
earlystopping = EarlyStopping()

tensorboard = TensorBoard(log_dir = "./logs/{}_{}".format(date, time()), 
                          histogram_freq = 0, 
                          write_graph = True, 
                          write_images = True)

train the model
the history object stores training data for later access, like plotting training curves

In [ ]:
history = sr.m.fit(X_train, 
                   to_categorical(Y_train), 
                   batch_size = batch_size, 
                   epochs = 25, 
                   verbose = 1, shuffle = True, 
                   class_weight = class_weights,
                   validation_data = (X_val, to_categorical(Y_val)), 
                   callbacks = [checkpointer]) # add more callbacks if you want